SQL databases store data in tables with a strict predened schema and follow the
ACID properties.

Atomicity - A transaction is all-or-nothing (it either completes fully or not at

Consistency – Data always remains valid and follows dened rules.

Isolation – Transactions don’t interfere with each other.


Durability – Once data is saved, it won’t be lost, even if the system crashes.


In [1]:
import redis

In [2]:
# Redis connection
redis_cache = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

In [3]:
redis_cache

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=localhost,port=6379,db=0)>)>)>

DATABASE INDEX:

In [ ]:
CREATE TABLE employees(
    id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    department VARCHAR(50),
    salary DECIMAL(20)
    
);

In [ ]:
CREATE INDEX idx_last_name ON employees (last_name);

In [ ]:
SELECT * FROM employees WHERE last_name = 'Smith';

In [ ]:
CREATE INDEX idx_full_name ON employees (first_name, last_name);

In [ ]:
CREATE TABLE users (
    user_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    last_login TIMESTAMP,
    status VARCHAR(20) DEFAULT 'active'
);

without index sql looking for every row

In [ ]:
INSERT INTO users (name, email, last_login, status) VALUES
('Alice', 'alice@example.com', '2025-04-10 10:00:00', 'active'),
('Bob', 'bob@example.com', '2025-04-12 15:30:00', 'active'),
('Charlie', 'charlie@example.com', '2025-04-11 09:20:00', 'inactive'),
('David', 'david@example.com', '2025-04-13 12:45:00', 'active'),
('Eve', 'eve@example.com', '2025-04-09 08:15:00', 'active');

In [ ]:
SELECT * FROM users WHERE email = 'alice@example.com';

In [ ]:
SELECT * FROM users WHERE status = 'active' AND last_login >= '2025-04-12 00:00:00' ORDER BY last_login DESC;

with index

In [ ]:
-- Index on email for fast lookups
CREATE INDEX idx_users_email ON users (email);

-- Composite index on status and last_login for activity queries
CREATE INDEX idx_users_status_last_login ON users (status, last_login);

idx_users_email: B-tree index on email for exact match queries.
idx_users_status_last_login: Composite index supporting filtering by status and sorting by last_login.

In [ ]:
SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'users';

In [ ]:
CREATE DATABASE index_demo;

USE index_demo;

CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    age INT,
    INDEX idx_email (email)   -- Create index on email
);
